In [6]:
import json, pycm
from tqdm import tqdm
from shroom_classifier import ShroomClassifier

In [7]:
def run_experiment(cp, filename):
    data = json.load(open(filename, "r"))
    for dp in tqdm(data):
        if "predicted" in dp:
            pass
        else:
            classification = cp.classify(dp["task"], dp["src"], dp["tgt"], dp["hyp"])
            dp["rationale"] = classification["rationale"]
            dp["predicted"] = classification["label"]
            json.dump(data, open(filename, "w+"))

In [8]:
cp = ShroomClassifier()
run_experiment(cp, 'results_val.model-agnostic.json')

100%|██████████| 499/499 [1:33:21<00:00, 11.23s/it]   
